## Library

In [ ]:
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Connect Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Modelling

### Load Data

In [ ]:
DATASET_DIR = "/content/drive/MyDrive/BANGKIT/dataset_min"

train_datagen = ImageDataGenerator(
  rotation_range=40,
  width_shift_range=0.2,
  height_shift_range=0.2,
  shear_range=0.2,
  zoom_range=0.2,
  horizontal_flip=True,
  fill_mode='nearest',
  validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
  directory=DATASET_DIR,
  seed=15,
  batch_size=20,
  subset='training',
  color_mode='rgb',
  class_mode='categorical',
  target_size=(256, 256)
)

validation_datagen = ImageDataGenerator(validation_split=0.2)

validation_generator = validation_datagen.flow_from_directory(
  directory=DATASET_DIR,
  seed=15,
  batch_size=20,
  subset='validation',
  color_mode='rgb',
  class_mode='categorical',
  target_size=(256, 256)
)

### Model

In [ ]:
efficientnet = tf.keras.applications.EfficientNetV2M(include_top=False, weights="imagenet",input_shape=(256, 256, 3))

for layer in efficientnet.layers:
  if layer.name == 'block7e_dwconv2':
    break
  layer.trainable = False

model = tf.keras.models.Sequential([
  efficientnet,
  tf.keras.layers.GlobalAveragePooling2D(),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(units=15, activation='softmax')
])

In [ ]:
model.compile(
  optimizer='rmsprop',
  loss='categorical_crossentropy',
  metrics=['accuracy']
)

In [ ]:
print(model.summary())

### Callback

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') > 0.97 and logs.get('val_accuracy') > 0.97):
      print("\nCancelling training!")
      self.model.stop_training = True

callbacks = myCallback()
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

### Train Model

In [ ]:
history = model.fit(
  train_generator,
  epochs=100,
  verbose=1,
  validation_data=validation_generator,
  callbacks=[callbacks, early_stop]
)

### Save Model

In [ ]:
model_name = "EfficientNetV2M"
acc = format(history.history['accuracy'][-21], '.4f')[2:]
val_acc = format(history.history['val_accuracy'][-21], '.4f')[2:]

filename = model_name + "_" + acc + "_" + val_acc + ".h5"
print(filename)

In [ ]:
model.save(filename)

### Model Peformance

In [ ]:
# Ambil nilai loss dan akurasi dari objek 'history'
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Visualisasi grafik loss
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(range(1, len(train_loss) + 1), train_loss, label='Training Loss')
plt.plot(range(1, len(val_loss) + 1), val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Visualisasi grafik akurasi
plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_acc) + 1), train_acc, label='Training Accuracy')
plt.plot(range(1, len(val_acc) + 1), val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Menampilkan grafik
plt.tight_layout()
plt.show()

## Predict

### Load Model

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/BANGKIT/Model/EfficientNetV2M_9889_9204.h5')

### Class Name

In [ ]:
classes = ['cendrawasih', 'geblek renteng', 'insang', 'kawung', 'lasem', 'mega mendung', 'nitik', 'parang', 'poleng', 'pring sedapur', 'sekar jagad', 'simbut', 'sogan', 'tambal', 'truntum']
print(classes)

### Predict Top 1

In [ ]:
img_width, img_height = 256, 256
img = image.load_img('36.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

pred =  model.predict(img)
pred_index = np.argmax(pred[0])
print(list(validation_generator.class_indices.keys())[pred_index])
print(pred[0][pred_index] * 100)

1/1 [==============================] - 8s 8s/step
geblek renteng
98.5456645488739


### Predict Top 3

In [ ]:
img_width, img_height = 256, 256
img = image.load_img('36.jpg', target_size = (img_width, img_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)

pred =  model.predict(img)

top_k_values, top_k_indices = tf.nn.top_k(pred, k=3)

for i in range(len(top_k_values[0])):
  print(classes[int(top_k_indices[0][i])] + ": " + str(float(top_k_values[0][i]) * 100))

1/1 [==============================] - 0s 42ms/step
parang: 99.99275207519531
tambal: 0.005792994488729164
sogan: 0.0013197688531363383
